<a href="https://colab.research.google.com/github/smturzo/rdkit_scripts/blob/main/RDKIT_MOLECULE_MANIPULATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install RDKit.
%%capture
!pip install rdkit

In [3]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
#from rdkit.Geometry import Transform3D
from rdkit.Geometry import rdGeometry as geom

from rdkit import DataStructs
import numpy as np
from scipy.spatial.transform import Rotation as R
from rdkit.Geometry import Point3D

def rotate_molecule(mol, axis_vector, angle_degrees):
    """
    Rotate coordinates around a given axis by a specified angle.

    :param coordinates: Array of XYZ coordinates (Nx3)
    :param axis_vector: Axis to rotate around
    :      for example -> x: [1, 0, 0], y: [0, 1, 0], z: [0, 0, 1]
    :                     xy:[1, 1, 0], yz:[0, 1, 1], xz:[1, 0, 1]
    :param angle_degrees: Rotation angle in degrees
    :return: Rotated coordinates (Nx3)
    """
    # Get Conformation:
    conf = mol.GetConformer()

    # Create rotation object
    rotation = R.from_rotvec(np.radians(angle_degrees) * np.array(axis_vector))

    # Apply rotation to coordinates
    for i in range(mol.GetNumAtoms()):
      pos = conf.GetAtomPosition(i)
      pos = np.array([pos.x, pos.y, pos.z])
      rotated_coords = rotation.apply(pos)
      conf.SetAtomPosition(i, Point3D(rotated_coords[0], rotated_coords[1], rotated_coords[2]))



# Create a Transform3D object
transform = Point3D()

# Create a translation vector
translation = np.array([0.0, 1.0, 0.0])#Point3D(0.0, 1.0, 0.0)

# Create benzene molecule
benzene = Chem.AddHs(Chem.MolFromSmiles('c1ccccc1'))
AllChem.EmbedMolecule(benzene)

# Convert benzene to XYZ format
def mol_to_xyz(mol):
    #mol = Chem.AddHs(mol)
    conf = mol.GetConformer()
    xyz = ""
    for atom in mol.GetAtoms():
        pos = conf.GetAtomPosition(atom.GetIdx())
        xyz += f"{atom.GetSymbol()} {pos.x:.4f} {pos.y:.4f} {pos.z:.4f}\n"
    return xyz

# Make a copy of the molecule
benzene_B = Chem.Mol(benzene)

# Translate the coordinates of the copied molecule
def translate_molecule(mol, translation_vector):
    conf = mol.GetConformer()
    for i in range(mol.GetNumAtoms()):
        pos = conf.GetAtomPosition(i)
        new_pos = pos + translation_vector
        conf.SetAtomPosition(i, new_pos)


translate_molecule(benzene_B, translation)

# Rotate molecule B by 30 degrees around the z-axis
rotate_molecule(benzene_B, [1, 0, 0], 90)

# Get XYZ formats
xyz_A = mol_to_xyz(benzene)
xyz_B = mol_to_xyz(benzene_B)


print(f"12\nXYZ format of molecule A:\n{xyz_A}12\nXYZ format of molecule B after translation:\n{xyz_B}")


12
XYZ format of molecule A:
C 0.7943 -1.1490 0.0150
C 1.3607 0.1298 -0.0022
C 0.5898 1.2681 -0.0170
C -0.7866 1.1450 -0.0149
C -1.3413 -0.1124 0.0019
C -0.5806 -1.2656 0.0170
H 1.4026 -2.0428 0.0267
H 2.4183 0.2113 -0.0036
H 1.0075 2.2686 -0.0304
H -1.4182 2.0293 -0.0265
H -2.4222 -0.2172 0.0037
H -1.0241 -2.2652 0.0304
12
XYZ format of molecule B after translation:
C 0.7943 -0.0150 -0.1490
C 1.3607 0.0022 1.1298
C 0.5898 0.0170 2.2681
C -0.7866 0.0149 2.1450
C -1.3413 -0.0019 0.8876
C -0.5806 -0.0170 -0.2656
H 1.4026 -0.0267 -1.0428
H 2.4183 0.0036 1.2113
H 1.0075 0.0304 3.2686
H -1.4182 0.0265 3.0293
H -2.4222 -0.0037 0.7828
H -1.0241 -0.0304 -1.2652

